In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import norm
import glob
import moviepy.editor as mpy
from mayavi import mlab

# Figure Options
mlab.init_notebook(width=700, height=700, backend = "x3d")
mlab.options.offscreen = True

Notebook initialized with x3d backend.


# Importing Data

In [2]:
dynamic_data= pd.read_csv("dynamic_results.csv")
result_length = int(max(dynamic_data.time)/0.025 + 1)
dynamic_grouped_time = dynamic_data.groupby('time')

In [3]:
# seperate runs 
dynamic_results_1 = dynamic_data.loc[0*result_length:1*result_length - 1]
dynamic_results_2 = dynamic_data.loc[1*result_length:2*result_length - 1]
dynamic_results_3 = dynamic_data.loc[2*result_length:3*result_length - 1]
dynamic_results_4 = dynamic_data.loc[3*result_length:4*result_length - 1]
dynamic_results_5 = dynamic_data.loc[4*result_length:5*result_length - 1]

# replace indexs
new_index = pd.Series(np.linspace(0, result_length-1, result_length), dtype = np.int32)

dynamic_results_1 = dynamic_results_1.set_index(new_index)
dynamic_results_2 = dynamic_results_2.set_index(new_index)
dynamic_results_3 = dynamic_results_3.set_index(new_index)
dynamic_results_4 = dynamic_results_4.set_index(new_index)
dynamic_results_5 = dynamic_results_5.set_index(new_index)

Selecting run to visulise:

In [4]:
sim_results = dynamic_results_1

# Functions for Visualisation

In [5]:
# System Parameters - from user guide
Lr = 0.085      # m - rotary arm length
Lp = 0.129      # m - pendulum length

In [6]:
def cylinder_plot(start_pos, end_pos, radius):

    #vector in direction of axis
    v = end_pos - start_pos

    #find magnitude of vector
    mag = norm(v)

    #unit vector in direction of axis
    v = v / mag

    #make some vector not in the same direction as v
    not_v = np.array([1, 0, 0])
    if (v == not_v).all():
        not_v = np.array([0, 1, 0])

    #make vector perpendicular to v
    n1 = np.cross(v, not_v)
    #normalize n1
    n1 /= norm(n1)

    #make unit vector perpendicular to v and n1
    n2 = np.cross(v, n1)

    #surface ranges over t from 0 to length of axis and 0 to 2*pi
    t = np.linspace(0, mag, 2)
    theta = np.linspace(0, 2 * np.pi, 100)
    rsample = np.linspace(0, radius, 50)

    #use meshgrid to make 2d arrays
    t, theta2 = np.meshgrid(t, theta)

    rsample,theta = np.meshgrid(rsample, theta)

    #generate coordinates for surface
    # "Tube"
    tube = [start_pos[i] + v[i] * t + radius * np.sin(theta2) * n1[i] + radius * np.cos(theta2) *       n2[i] for i in [0, 1, 2]]
    # "Bottom"
    bottom = [start_pos[i] + rsample[i] * np.sin(theta) * n1[i] + rsample[i] * np.cos(theta) * n2[i] for i in [0, 1, 2]]
    # "Top"
    top = [start_pos[i] + v[i]*mag + rsample[i] * np.sin(theta) * n1[i] + rsample[i] * np.cos(theta) * n2[i] for i in [0, 1, 2]]

    return tube, bottom, top

In [7]:
def pendulum_sim_plot(theta, alpha, file_name):
    
    # Base Object
    base_arm_height = 0.015
    base_p0 = np.array([0, 0, 0])
    base_p1 = np.array([0, 0, base_arm_height])
    base_r  = 0.015
    base_c, base_b, base_t = cylinder_plot(base_p0, base_p1, base_r)

    # Rotational Arm
    arm_p0 = np.array([0, 0, base_arm_height/2])
    arm_p1 = np.array([Lr*np.cos(theta), Lr*np.sin(theta), base_arm_height/2])
    arm_r  = 0.005
    arm_c, arm_b, arm_t = cylinder_plot(arm_p0, arm_p1, arm_r)

    # Pendulum Arm
    pend_p0 = np.array([Lr*np.cos(theta)*(1-0.005), Lr*np.sin(theta)*(1-0.005), base_arm_height/2])
    pend_p1 = pend_p0 + np.array([-Lp*np.sin(theta)*np.sin(alpha), Lp*np.cos(theta)*np.sin(alpha), Lp*np.cos(alpha)-0.02])
    pend_r  = 0.003
    pend_c, pend_b, pend_t = cylinder_plot(pend_p0, pend_p1, pend_r)
    
    # Plotting
    fig = mlab.figure(bgcolor=(1, 1, 1), fgcolor=(0.5, 0.5, 0.5))
    mlab.clf()

    # Plot Arm
    mlab.mesh(arm_c[0], arm_c[1], arm_c[2], color = (0.5,0.5,0.5))
    mlab.mesh(arm_b[0], arm_b[1], arm_b[2], color = (0.5,0.5,0.5))
    mlab.mesh(arm_t[0], arm_t[1], arm_t[2], color = (0.5,0.5,0.5))

    # Plot Base
    mlab.mesh(base_c[0], base_c[1], base_c[2], color = (0.2,0.2,0.2))
    mlab.mesh(base_b[0], base_b[1], base_b[2], color = (0.2,0.2,0.2))
    mlab.mesh(base_t[0], base_t[1], base_t[2], color = (0.2,0.2,0.2))

    # Plot Pendulum
    mlab.mesh(pend_c[0], pend_c[1], pend_c[2], color = (0.8,0.2,0))
    mlab.mesh(pend_b[0], pend_b[1], pend_b[2], color = (0.8,0.2,0))
    mlab.mesh(pend_t[0], pend_t[1], pend_t[2], color = (0.8,0.2,0))
    
    mlab.view(60, 60, 0.6, focalpoint=(0,0,0))
    
    # Save File
    mlab.savefig(f'animation/{file_name}.png')
    
    return


# Creating Animation File

Sometimes it doesn't complete the entire range in one go so I just adjust the numbers to what the last image generated is, not sure if theres an implementation problem or if my computer is too slow ????

In [8]:
for i in range(170, len(sim_results)):
    pendulum_sim_plot(sim_results.theta[i], sim_results.alpha[i], f'anim_{i}')

In [9]:
file_name = 'simulationAnimation'
fps = 40
file_list = glob.glob('animation/*.png')
list.sort(file_list, key=lambda x: int(x.split('_')[1].split('.png')[0]))
clip = mpy.ImageSequenceClip(file_list, fps=fps)
clip.set_duration(5).write_videofile('animation/{}.mp4'.format(file_name))
clip.write_gif('animation/{}.gif'.format(file_name), fps=fps)

Moviepy - Building video animation/simulationAnimation.mp4.
Moviepy - Writing video animation/simulationAnimation.mp4



t:   3%|█                            | 7/202 [00:00<00:02, 69.30it/s, now=None]

Moviepy - Done !
Moviepy - video ready animation/simulationAnimation.mp4
MoviePy - Building file animation/simulationAnimation.gif with imageio.
